<a href="https://colab.research.google.com/github/rikharigaurav/Pytorch/blob/main/Pytorch_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f   #for functions that don't have any parameters
from torch.utils.data import DataLoader  # Gives easier dataset management and creates mini batches
import torchvision.datasets as datasets
import torchvision.transforms as transforms #Transformation we can perform on our dataset


In [13]:
#Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
#Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 32
num_epochs = 2

In [15]:
# Create a bidirectional LSTM
class BLSTM(nn.Module):
  def __init__(self,input_size, hidden_size, num_layers, num_classes):
    super(BLSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional =True)
    self.fc = nn.Linear(hidden_size * 2, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)

    out, _ = self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1, :])

In [16]:
#Load Data
train_dataset = datasets.MNIST(root = 'dataset/', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle =True)
test_dataset = datasets.MNIST(root = 'dataset/', train = True, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle =True)

In [17]:
#Initialize network
model = BLSTM(input_size, hidden_size, num_layers, num_classes).to(device)

In [24]:
#Loss and optimizer
Loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [25]:
# Train Network
model.train()
for epoch in range(num_epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    data = data.to(device = device).squeeze(1)
    target = target.to(device = device)

    #forward
    predict = model(data)
    loss = Loss(predict, target)

    #backward
    optimizer.zero_grad()
    loss.backward()

    #gradient descent or adam step
    optimizer.step()

# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")
  correct = 0
  samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device).squeeze(1)
      y = y.to(device)

      predict = model(x)
      _, prediction = predict.max(1)
      correct += (prediction == y).sum()
      samples += prediction.size(0)

    print(f"Got {correct}/{samples} with accuracy {float(correct) / float(samples) * 100:.2f}")

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not NoneType